In [91]:
import pandas as pd
import statsmodels.api as sm

tommy_johns = pd.read_csv('Tommy_John_Surgery_List.csv', skiprows = 1, usecols = ["Player", "TJ Surgery Date"])
pitch_velocity = pd.read_csv('savant_data_pitch_velocity.csv', usecols = ["player_name", "velocity", "total_pitches"])
tommy_johns = tommy_johns.dropna(how = 'all')


tommy_johns['Player'] = tommy_johns['Player'].apply(lambda x: ', '.join(x.split()[::-1]))

result = pd.merge(pitch_velocity, tommy_johns, how = 'left', left_on = ['player_name'], right_on = ['Player'])

result['TJ'] = result['TJ Surgery Date'].notna().astype(int)
result = result[['player_name', 'velocity', 'TJ', 'total_pitches']]
result = result[result['total_pitches']>1000]

model = sm.Logit(result['TJ'], result[['velocity', 'total_pitches']]).fit()
model.summary()




Optimization terminated successfully.
         Current function value: 0.650650
         Iterations 4


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                     TJ   No. Observations:                 1780
Model:                          Logit   Df Residuals:                     1778
Method:                           MLE   Df Model:                            1
Date:                Mon, 20 May 2024   Pseudo R-squ.:               -0.001236
Time:                        23:22:05   Log-Likelihood:                -1158.2
converged:                       True   LL-Null:                       -1156.7
Covariance Type:            nonrobust   LLR p-value:                     1.000
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
velocity         -0.0062      0.001     -8.436      0.000      -0.008      -0.005
total_pitches -3.608e-06   7.54e-06     -0.478      0.632   -1.84e-05    1.12e-05
=================================================================================
"""

In [94]:
import pandas as pd
import matplotlib.pyplot as plt

tommy_johns = pd.read_csv('Tommy_John_Surgery_List.csv', skiprows = 1, usecols = ["TJ Surgery Date"])
tommy_johns = tommy_johns.dropna(how = 'all')

result = 